In [13]:
import random
import numpy as np

In [14]:
# Defines the structure of the graph and connects the nodes within the graph
class Graph:
    def __init__(self, graph_dict):
        self.graph_dict = graph_dict
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.connect(b, a, dist)
                self.connect(a, b, dist)

    def connect(self, A, B, distance):
        self.graph_dict.setdefault(A, {})[B] = distance

In [15]:
# Defines a node within the graph
class Node:
    def __init__(self, state):
        self.state = state

In [16]:
# Defines the structure of the Traveling Sales Person specific graph
class TSP():
    def __init__(self, initial_state, distances):
        self.initial_state = initial_state
        self.distances = distances
        
    # Defines the actions an agent can take - ie the edges it can traverse
    def action(self, state):
        neighbor = state[:]
        left = random.randint(0, len(neighbor) - 1)
        right = random.randint(0, len(neighbor) - 1)
        if left > right:
            left, right = right, left
        neighbor[left: right + 1] = reversed(neighbor[left: right + 1])
        return neighbor

    # Returns the result of an action taken by the agent
    def result(self, state, action):
        return action(state)

    # Calculates the cost of traversing an edge
    def path_cost(self, state1, state2):
        cost = 0
        for i in range(len(state2) - 1):
            cost += self.distances[state2[i]][state2[i + 1]]
        cost += self.distances[state2[0]][state2[-1]]
        return cost

    def value(self, state):
        return -1 * self.path_cost(None, state)

In [17]:
# Chooses which iteration of the algorithm has produced the best result
def argmax_random_tie(neighbors, key=lambda x: x):
    max_val = max((key(x) for x in neighbors))
    max_items = [x for x in neighbors if key(x) == max_val]
    return random.choice(max_items)

In [18]:
# Defines the hill climbing algorithm
def hill_climbing(tsp):
    # Finds the neighboring nodes it is possible for the agent to reach   
    def find_neighbors(state, num_neighbors):      
        neighbors = []
        
        for i in range(num_neighbors):
            new_state = tsp.action(state)
            neighbors.append(Node(new_state))
            state = new_state
            
        return neighbors

    # Max iterations
    itr = 10000
    
    # Initialize problem
    current_node = Node(tsp.initial_state)

    # Hill Climbing algorithm implementation
    for _ in range(itr):
        # Finds neighbors
        neighbors = find_neighbors(current_node.state, 100)
        if neighbors:
            # Finds best neighbor to visit
            best_neighbor = argmax_random_tie(neighbors, key=lambda node: tsp.value(node.state))
            if tsp.value(best_neighbor.state) <= tsp.value(current_node.state):
                current_node = best_neighbor

        else:
            return current_node.stat
        
    return current_node.state


In [58]:
# Defines the genetic algorithm
def genetic_algorithm(tsp, population_size=100, generations=1000, mutation_rate=0.1):
    # Defines the crossover function for the genetic algorithm
    def crossover(parent1, parent2):
        n = len(parent1)
        start, end = sorted(random.sample(range(n), 2))
        child = [-1] * n
        child[start:end] = parent1[start:end]
        
        index = end
        for city in parent2[end:] + parent2[:end]:
            if city not in child:
                child[index % n] = city
                index += 1

        return child
    
    # Defines the mutations for the genetic algroithm
    def mutate(solution):
        n = len(solution)
        index1, index2 = sorted(random.sample(range(n), 2))
        solution[index1], solution[index2] = solution[index2], solution[index1]
        return solution

    population = [Node(tsp.initial_state) for _ in range(population_size)]

    for _ in range(generations):
        # Choose the parents for the genetic algorithm
        parents = [random.choice(population) for _ in range(population_size)]

        # Generate the children
        next_generation = []
        for i in range(0, population_size, 2):
            parent1 = parents[i]
            parent2 = parents[i + 1]
            child1 = crossover(parent1.state, parent2.state)
            child2 = crossover(parent2.state, parent1.state)
            next_generation.extend([Node(mutate(child1)), Node(mutate(child2))])

        population = argmax_random_tie(next_generation, key=lambda node: tsp.value(node.state))
        population = [Node(population.state)]

    # Choose best solution
    best_solution = argmax_random_tie(population, key=lambda node: tsp.value(node.state))
    return best_solution.state

In [52]:
# Defines the map of romania used in the graph for the traveling sales person problem
romania_map = Graph(dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142)))

romania_map.locations = dict(
    Arad=(91, 492), Bucharest=(400, 327), Craiova=(253, 288),
    Drobeta=(165, 299), Eforie=(562, 293), Fagaras=(305, 449),
    Giurgiu=(375, 270), Hirsova=(534, 350), Iasi=(473, 506),
    Lugoj=(165, 379), Mehadia=(168, 339), Neamt=(406, 537),
    Oradea=(131, 571), Pitesti=(320, 368), Rimnicu=(233, 410),
    Sibiu=(207, 457), Timisoara=(94, 410), Urziceni=(456, 350),
    Vaslui=(509, 444), Zerind=(108, 531))

In [53]:
# Initialize the tsp problem
distances = {}
cities = []

for city in romania_map.locations.keys():
    distances[city] = {}
    cities.append(city)


In [54]:
# Initialize the tsp problem
for city1, coords1 in romania_map.locations.items():
        for city2, coords2 in romania_map.locations.items():
            distances[city1][city2] = np.linalg.norm(
                [coords1[0] - coords2[0], coords1[1] - coords2[1]])
            distances[city2][city1] = np.linalg.norm(
                [coords1[0] - coords2[0], coords1[1] - coords2[1]])

In [55]:
# Initialize the tsp problem
tsp = TSP(cities, distances)

In [56]:
# Run the hill climbing algorithm
final_state = hill_climbing(tsp)
print(final_state)
total_cost = tsp.path_cost(tsp.initial_state, final_state)
print(total_cost)

['Zerind', 'Lugoj', 'Timisoara', 'Pitesti', 'Neamt', 'Bucharest', 'Giurgiu', 'Hirsova', 'Mehadia', 'Oradea', 'Fagaras', 'Craiova', 'Rimnicu', 'Arad', 'Urziceni', 'Drobeta', 'Vaslui', 'Iasi', 'Sibiu', 'Eforie']
4686.23640900363


In [59]:
# Run the genetic algorithm
final_state = genetic_algorithm(tsp)
print(final_state)
total_cost = tsp.path_cost(tsp.initial_state, final_state)
print(total_cost)

['Arad', 'Zerind', 'Oradea', 'Bucharest', 'Urziceni', 'Hirsova', 'Eforie', 'Vaslui', 'Iasi', 'Neamt', 'Fagaras', 'Sibiu', 'Lugoj', 'Mehadia', 'Drobeta', 'Craiova', 'Giurgiu', 'Pitesti', 'Rimnicu', 'Timisoara']
2002.6010047358545
